## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [2]:
# Global variables

# Vocabulary
vocab_size = 3000
context_size = 9
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 32
epochs = 10
lr = 0.01

# Model
embedding_dim = 64
hidden_dim = 128

## Faz download e carrega o dataset

In [3]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [4]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [5]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [6]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

O instincto selvagem suppria a industria do homem civilisado; a primeira das artes foi incontestavelmente a arte da guerra,--a arte da defeza e da vingança, os dous mais fortes estímulos do coração humano.

--De um crime, irmão. Dai-me luz; vou escrever a Fr. Diogo do Rosario, nosso prior, porque de onde vou talvez não volte, nem tenhais mais novas de mim.

O indio, emquanto se passava a rapida scena que descrevemos, reflectia profundamente.

--E se a pedra quizer fazer mal á senhora quem a defenderá? perguntou a menina sorrindo e fazendo allusão á narração do indio.

Quando o escudeiro voltou a si da vertigem, uma rodilho de cipós ligava-o ao tronco desde o joelho até os hombros; o indio seguira seu caminho placidamente.

Number of paragraphs: 4892


4892

## Análise do dataset

In [7]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [8]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [9]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 3000


In [10]:
#pattern = r'\w+'

def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 0, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [11]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    # Define your data here
    self.context = context
    self.removed = 0
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):

    tokens = []
    targets = []
    for paragraph in paragraphs:
      encoded = encode_sentence(paragraph, vocab)
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      # Do not add examples with unknown tokens
      if 0 not in encoded:
        for i in range(len(encoded) - self.context):
          tokens.append(encoded[i:i+self.context])
          targets.append(encoded[i+self.context])
      else:
        self.removed+=1

    return tokens, targets


In [12]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

print("Training samples: " + str(len(train_data)))
print("Validation samples: " + str(len(val_data)))

Training samples: 3913
Validation samples: 979


In [13]:
train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Samples
print("Training samples:")
print(train_dataset[:5])
print()
print("Validation samples:")
print(val_dataset[:5])
print()
print("Removed training samples: " + str(train_dataset.removed))
print("Removed validation samples: " + str(val_dataset.removed))

Training samples:
(tensor([[  41,   74,   66,  544,   53,    5,  745,   34, 1066],
        [  74,   66,  544,   53,    5,  745,   34, 1066, 1067],
        [  66,  544,   53,    5,  745,   34, 1066, 1067,    2],
        [ 544,   53,    5,  745,   34, 1066, 1067,    2, 1256],
        [  53,    5,  745,   34, 1066, 1067,    2, 1256,    1]]), tensor([1067,    2, 1256,    1,  140]))

Validation samples:
(tensor([[  41,   74,   66,  544,   53,    5,  745,   34, 1066],
        [  74,   66,  544,   53,    5,  745,   34, 1066, 1067],
        [  66,  544,   53,    5,  745,   34, 1066, 1067,    2],
        [ 544,   53,    5,  745,   34, 1066, 1067,    2, 1256],
        [  53,    5,  745,   34, 1066, 1067,    2, 1256,    1]]), tensor([1067,    2, 1256,    1,  140]))

Removed training samples: 3423
Removed validation samples: 3423


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[   7,    1,    1,    1,  173,    5,  469,   16,    3],
        [ 719,  151,  544,  331,  282,   41,  506,   53, 1257],
        [1490,   15,  812,   10, 2249,    6, 1942, 1239,   22],
        [  39,  257,   29, 2379,  631,  324,   71,  721,    2],
        [  12,   37,   46,  272,   34,  422,   31,   17,  410],
        [   1,   66,    1,  747,    1,  341,   92,  332,  259],
        [ 171,    3,   58,    9,   39, 1236, 1127, 1190,   18],
        [  40,    2, 1612,    3, 2605,   11,    4,    8,  203],
        [  17,  172,  135,   10,  245,    6, 1056,    7,  589],
        [ 949,    9,    3,  398, 1997, 2385,    2,  171,    5],
        [  17,  179,    2,  677,  295,   15, 1588,   42, 2341],
        [  44, 1131,    9,  858,   38,  309,    9,   37,    3],
        [1945,  148,  151,  544,   90, 1512,  545,  746,    1],
        [ 178,   20,  118,  172,    9,   93,   46,  540,    6],
        [  89,  371,   13,  504,    6,   35,    2,   12,   37],
        [  30,    1,   47,    6,   73, 

## Model

In [15]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, vocab_size+1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer with Relu activation
        out = self.linear1(embeds)
        out = F.relu(out)
        # Second layer
        out = self.linear2(out)
        return out

In [16]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [17]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([32, 9])
torch.Size([32])


In [18]:
output = model(input)

In [19]:
output.argmax(dim=1)

tensor([1284, 1305, 1305, 1370, 1298, 2364, 1664, 1499, 2051,  227,  478,  979,
        2120, 1305, 2992, 2667, 2364, 1974,  478, 1378, 2050, 1602, 1664, 2364,
        1742, 1602, 2364, 1688,  478,  992, 2045,  284])

In [20]:
target

tensor([ 703,    1, 1310, 1591,  529,   62,    7,   19,   23,   17,    5,  189,
          92,    9,    7,    7,    7,    7,   52,    9,  418,  557, 2936,    5,
         110,    7,    1,    3,  265,   79,  116,    3])

## Training

In [21]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [22]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=576, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3001, bias=True)
)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 653,049 parâmetros.


In [24]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.0420
Initial Perplexity: 3108.8982


In [25]:
# Training Loop
model.train()
for epoch in range(epochs):

  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/10], Loss: 6.8823, Accuracy: 0.08%, Perplexity: 974.9133
Epoch [2/10], Loss: 4.7271, Accuracy: 0.18%, Perplexity: 112.9706
Epoch [3/10], Loss: 3.3866, Accuracy: 0.35%, Perplexity: 29.5663
Epoch [4/10], Loss: 2.3061, Accuracy: 0.52%, Perplexity: 10.0356
Epoch [5/10], Loss: 1.5323, Accuracy: 0.67%, Perplexity: 4.6289
Epoch [6/10], Loss: 1.0285, Accuracy: 0.77%, Perplexity: 2.7970
Epoch [7/10], Loss: 0.6556, Accuracy: 0.85%, Perplexity: 1.9264
Epoch [8/10], Loss: 0.4645, Accuracy: 0.90%, Perplexity: 1.5912
Epoch [9/10], Loss: 0.3410, Accuracy: 0.93%, Perplexity: 1.4064
Epoch [10/10], Loss: 0.2573, Accuracy: 0.95%, Perplexity: 1.2935


## Avaliação

In [26]:
model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0
eval_round = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)        
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()
        eval_round += 1

# Calculate accuracy
acc = 100 * correct_sum/total_sum

# Calculate average perplexity
average_loss = loss_sum / eval_round
average_perplexity = torch.exp(average_loss)

print(f'Test Accuracy: {acc:.2f}%')
print(f'Average Loss: {average_loss:.2f}')
print(f'Average Perplexity: {average_perplexity:.2f}')

Test Accuracy: 96.90%
Average Loss: 0.16
Average Perplexity: 1.17


## Exemplo de uso

In [27]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")

context_size = 9
max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase: --Tendes mais experiencia do que eu, Sr. cavalheiro; mas queira Deus que vos enganeis.
[428, 46, 2042, 11, 4, 88, 2, 180, 1]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367, 79]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367, 79, 3]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367, 79, 3, 72]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367, 79, 3, 72, 215]
[428, 46, 2042, 11, 4, 88, 2, 180, 1, 141, 9, 7, 3, 2367, 79, 3, 72, 215, 3]
['tendes', 'mais', 'experiencia', 'do', 'que', 'eu', ',', 'sr', '.'] ['coração', ';', 'e', 'a', 'presas', 'mesmo', 'a', 'já', 'braço', 'a'] 
